<a href="https://colab.research.google.com/github/dylanbforde/ai-medical-image-analysis/blob/main/MRNet_tutorial_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TO DO:** Make a copy of this notebook in your own Google drive and edit the copy.

**TO DO:** Download the data at the following link https://stanfordmlgroup.github.io/competitions/mrnet/ and upload it to your Google Drive.

This will download a folder named 'data'.

*   The dataset consists of 1,250 knee MRIs with image level labels.
*   The training data consists of 1,130 MRIs and the validation data consists of 120 MRIs.
*   They are labelled as abnormal, having an acl tear and/or meniscus tear.
*   Each MRI exam includes data from the axial, coronal and sagittal plane.
*   Axial is a Proton-Density series, coronal is a T1-weighted series and sagittal is T2-weighted series.






Go to "Edit" on the toolbar, then "Notebook Settings" and change the hardware accelerator to GPU.


#Mount Google Drive to access your data



In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


The following code uses a python library named 'torchsample'. This is not installed in Google Colab. We can import it by running the commands in the following cell. The exclamation mark communicates to Google Colab to run the commands in the terminal rather than in Python in the current notebook.


In [2]:
!pip install torchsample
!pip install visdom
!pip install nibabel
!pip install h5py
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.6/393.6 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torchsample: filename=torchsample-0.1.0-py3-none-any.whl size=16698 sha256=721547ae177a1b6b82366f2a6f606e4739071ec41b641ff2953153d69becef69
  Stored in directory: /root/.cache/pip/wheels/c6/c8/b6/19457b219a37c931e360e7bce0e014e92422de1f6257d1a49d
Successfully built torchsample
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408288 sha256=73e4e8d7736a34e7946b5129a69e06cebd61ee747d3e55f83b2698bf6461e124
  Stored in directory: /root/.cache/pip/wheels/fa/a4/bb/2be445c295d88a74f9c0a4232f04860ca489a5c7c57eb959d9
Successfully 

In [3]:
#import all libraries
import torch.optim as optim
import torch
import torch.nn as nn
from torchvision import models
import numpy as np
import os
import sys
import pickle
import torch.nn.functional as F
import torch.utils.data as data
import pandas as pd
from torch.autograd import Variable
from torchvision import transforms
from tensorboardX import SummaryWriter
import math
from sklearn import metrics

#Define your model
The model is defined in the class 'Net'. The 'init' function initialises the architecture of the model.

The line of code; ```self.pretrained_model = models.resnet18(pretrained=True)``` initialises a pre-trained ResNet18, pre-trained on the ImageNet Dataset. This initialises the weights of the model with the weights for a ResNet18 model that was trained on the ImageNet dataset. This speeds up training.

The line of code ```self.classifer = nn.Linear(1000, 1)``` is a fully connected layer that makes the final prediction.

After the model is initialised, the forward function is called iteratively throughout the training process. The output size of each line is shown in the code.

More information con defining models can be found at https://pytorch.org/vision/stable/models.html

In [4]:
models.resnet18(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 174MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
list( models.resnet18(pretrained=True).children())[:-1]

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   )
   (1): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), pad

In [6]:
#modify the last fully connected layer to output (1) instead of (1000)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained_model =  nn.Sequential(*list(models.resnet18(pretrained=True).children())[:-1]  )    # delete the last fc layer.
        self.classifer = nn.Linear(512, 1)


    def forward(self, x):
        # input size of x (1, s, 3, 256, 256) where s is the number of slices in one MRI
        x = torch.squeeze(x, dim=0) #output size (s, 3, 256, 256)
        x = self.pretrained_model(x) #output size (s, 512)
        output = torch.max(x, 0, keepdim=True)[0] #output size (1, 512)
        output = self.classifer(output.squeeze(2).squeeze(2)) #output size (1)

        return output

In [7]:
#add another fully connected layer to convert output (1,1000) to (1)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained_model =  models.resnet18(pretrained=True)   # delete the last fc layer.
        self.classifer = nn.Linear(1000, 1)


    def forward(self, x):
        # input size of x (batch_size, s, 3, 256, 256) where s is the number of slices in one MRI
        batch_size, num_slices, channels, height, width = x.size()
        # Reshape the input to process slices individually
        x = x.view(-1, channels, height, width) # output size (batch_size * num_slices, 3, 256, 256)

        x = self.pretrained_model(x) # output size (batch_size * num_slices, 1000)
        x = x.view(batch_size, num_slices, -1) # output size (batch_size, num_slices, 1000)

        output = torch.max(x, 1, keepdim=True)[0] # output size (batch_size, 1, 1000)
        output =nn.ReLU()(output)
        output = self.classifer(output.squeeze(1)) # output size (batch_size, 1)

        return output

In [8]:
mod=Net()
mod

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Net(
  (pretrained_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

TO NOTE:
Models defined in Pytorch expect 2D image data in the dimensions (batch size, channels (colours), height of the image, width of the image)

#Create Dataloader
The 'init' function initialises the dataloader. This class is responsible for loading the datasets. It takes the 'root_dir', 'task', 'plane', 'train' and 'transform' as input parameters.
root_dir - the directory to where the data is stored.

task - whether the model is being trained to detect acl tears, meniscus tears or abnormalities. Possible values are 'acl', 'meniscus' or 'abnormal'.

plane - whether the model is being trained on axial, coronal or sagittal data. Possible values are 'axial', 'coronal' or 'sagittal'.

train - is this the dataloader for the training data or the validation data. Possible values are 'True' to load training data or 'False' to load validation data.

transform - a compose function for performing transformations to the images.

The init function creates 1) a list of paths to each MRI, 2) a corresponding list of labels that are either ones or zeros and 3) weights.


---



The __len__ function returns the length of the dataset.


---
The __getitem__ function is iteratively called throughout the training process. It takes an index as a input parameter. It loads the MRI at the given index from the list of paths defined in the init function. It also returns the label and weight for the MRI at that index.



In [9]:
class Dataset(data.Dataset):
    def __init__(self, root_dir, task, plane, train=False, transform=None):
        super().__init__()
        self.task = task
        self.plane = plane
        self.root_dir = root_dir
        self.train=train
        if self.train == True:
            self.folder_path = self.root_dir + 'train/{0}/'.format(plane)
            self.records = pd.read_csv(
                self.root_dir + 'train-{0}.csv'.format(task), header=None, names=['id', 'label'])
        else:
            self.folder_path = self.root_dir + 'valid/{0}/'.format(plane)

            self.records = pd.read_csv(
                self.root_dir + 'valid-{0}.csv'.format(task), header=None, names=['id', 'label'])

        self.records['id'] = self.records['id'].map(
            lambda i: '0' * (4 - len(str(i))) + str(i))
        self.paths = [self.folder_path + filename +
                      '.npy' for filename in self.records['id'].tolist()]
        self.labels = self.records['label'].tolist()

        self.transform = transform

        pos = np.sum(self.labels)
        neg = len(self.labels) - pos
        self.weights = [1, neg / pos]


    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        array = np.load(self.paths[index])

        label = self.labels[index]
        label = torch.FloatTensor([label])

        if self.transform:
          transformed_slices = []
          for i in array:
            transformed_slice = self.transform(i)
            transformed_slice = transformed_slice.repeat(3, 1, 1)
            transformed_slices.append(transformed_slice)
          array = torch.stack(transformed_slices)
        else:
          array = torch.from_numpy(array).float()
          array = array.unsqueeze(1)
          array = array.repeat(1, 3, 1, 1)

        array = array.float()

        if label.item() == 1:
            weight = np.array([self.weights[1]])
            weight = torch.FloatTensor(weight)
        else:
            weight = np.array([self.weights[0]])
            weight = torch.FloatTensor(weight)

        if array.shape[0] < 32:
            zeros = torch.zeros((32 - array.shape[0], 3, 256, 256))
            array = torch.cat((array, zeros), 0)

        elif array.shape[0] > 32:
            array = array[:32, :, :, :]

        return array, label, weight

#Train the model
##Define variables
**TO DO:** Change directory to where you store your data. Use the toolbar to the side of this page to view your file system.

In [10]:
directory = "/content/drive/Shared drives/MRNet Group Assignment/MRI Data/"
task = 'acl'
plane = 'sagittal'
lr = 1e-5 #learning rate
num_epochs = 50 # number of epochs

##Initialise the model, optimiser, scheduler, transformations and data loader.

In [11]:

model = Net() #initialise the model
if torch.cuda.is_available(): #if there is a GPU available, put the model on the GPU
    model = model.cuda()

optimizer = optim.Adam(model.parameters(), lr= lr, weight_decay=0.1) #define the optimiser as Adam

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, patience=4, factor=.3, threshold=1e-4, verbose=True) #define a scheduler that decreases the learning rate if there has been no reduction in validation loss is four epochs

#define a compose function that is a series of transformations on the images.
augmentor = transforms.Compose([
    # 1. numpy / PIL  ➜  float32 tensor in [0, 1]
    transforms.ToTensor(),

    # 2. ± 25° random rotation (keeps image size)
    transforms.RandomRotation(25),

    # 3. Random translation: up to 11 % of width & height
    transforms.RandomAffine(
        degrees=0,                 # no extra rotation here
        translate=(0.11, 0.11)
    ),

    # 4. Random left-right flip
    transforms.RandomHorizontalFlip()
])

#initialise the train and validation datasets (class we defined earlier) and then initialise a Pytorch's dataloader
train_dataset = Dataset(directory, task, plane,
                         train=True, transform=augmentor)
valid_dataset = Dataset(
      directory, task, plane, train=False, transform = None)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=10, shuffle=True, num_workers=2, drop_last=False)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=10, shuffle=False, num_workers=2, drop_last=False)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


##Training Loop

In [ ]:
import logging

# Configure logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s') # Removed logging configuration

early_trigger = 10 #if the validation AUC hasn't increased in ten epochs, stop the training
early_stop = 0 #counter for the number of iterations where there has been no increase in validation AUC
best_val_auc = 0
#for loop for each epoch
for epoch in range(num_epochs):
      print(f"Starting epoch {epoch+1}/{num_epochs}")
      #get learning rate
      current_lr = lr

      y_preds = []
      y_trues = []
      losses = []
      _ = model.train()
      print("Starting training phase") # Changed to print
      #loop through each MRI in the training set
      for i, (image, label, weight) in enumerate(train_loader):
          print(f"Training batch {i+1}/{len(train_loader)}") # Changed to print
          optimizer.zero_grad()

          #load all data onto the GPU
          if torch.cuda.is_available():
              image = image.cuda()
              label = label.cuda()
              weight = weight.cuda()

          #pass the MRI through the model
          prediction = model.forward(image.float())

          #calculate the loss
          loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction, label)
          loss.backward() #back propagation
          optimizer.step()

          loss_value = loss.item()
          losses.append(loss_value)

          probas = torch.sigmoid(prediction) #convert output of model (logits) to a value between zero and one. This can be interpretted as a probability

          y_trues.extend(label.cpu().tolist())
          y_preds.extend(probas.cpu().tolist())

          try:
              auc = metrics.roc_auc_score(y_trues, y_preds)
          except:
              auc = 0.5

          train_loss = np.round(np.mean(losses), 4)
          train_auc = np.round(auc, 4)

      #evaluate the model on the validation data after each epoch
      _ = model.eval()
      y_trues = []
      y_preds = []
      losses = []
      print("Starting validation phase") # Changed to print
      for i, (image, label, weight) in enumerate(valid_loader):
        print(f"Validation batch {i+1}/{len(valid_loader)}") # Changed to print
        if torch.cuda.is_available():
            image = image.cuda()
            label = label.cuda()
            weight = weight.cuda()


        prediction = model.forward(image.float())

        loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction, label)

        loss_value = loss.item()
        losses.append(loss_value)

        probas = torch.sigmoid(prediction)

        y_trues.extend(label.cpu().tolist())
        y_preds.extend(probas.cpu().tolist())

        try:
            auc = metrics.roc_auc_score(y_trues, y_preds)
        except:
            auc = 0.5

        val_loss = np.round(np.mean(losses), 4)
        val_auc = np.round(auc, 4)

      if val_auc > best_val_auc:
        best_val_auc = val_auc
        early_stop=0
      else:
        early_stop+= 1

      if early_stop == early_trigger:
        print('Early stopping after {} epochs'.format(epoch))
        sys.exit()
      scheduler.step(val_loss)

      print("epoch : {0} | train loss : {1} | train auc {2} | val loss {3} | val auc {4} ".format(
          epoch, train_loss, train_auc, val_loss, val_auc))


      print('-' * 30)

Starting epoch 1/50
Starting training phase
Training batch 1/113
Training batch 2/113
Training batch 3/113
Training batch 4/113
Training batch 5/113
Training batch 6/113
Training batch 7/113
Training batch 8/113
Training batch 9/113
Training batch 10/113
Training batch 11/113
Training batch 12/113
Training batch 13/113
Training batch 14/113
Training batch 15/113
Training batch 16/113
Training batch 17/113
Training batch 18/113
Training batch 19/113
Training batch 20/113
Training batch 21/113
Training batch 22/113
Training batch 23/113
Training batch 24/113
Training batch 25/113
Training batch 26/113
Training batch 27/113
Training batch 28/113
Training batch 29/113
Training batch 30/113
Training batch 31/113
Training batch 32/113
Training batch 33/113
Training batch 34/113
Training batch 35/113
Training batch 36/113
Training batch 37/113
Training batch 38/113
Training batch 39/113
Training batch 40/113
Training batch 41/113
Training batch 42/113
Training batch 43/113
Training batch 44/1

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Validation batch 2/12


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Validation batch 3/12


...

Validation batch 3/12


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Validation batch 4/12


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Validation batch 5/12
Validation batch 6/12
Validation batch 7/12
Validation batch 8/12
Validation batch 9/12
Validation batch 10/12
Validation batch 11/12
Validation batch 12/12
epoch : 2 | train loss : 0.9387 | train auc 0.7767 | val loss 27.1689 | val auc 0.6765 
------------------------------
Starting epoch 4/50
Starting training phase
Training batch 1/113
Training batch 2/113
Training batch 3/113
Training batch 4/113
Training batch 5/113
Training batch 6/113
Training batch 7/113
Training batch 8/113
Training batch 9/113
Training batch 10/113
Training batch 11/113
Training batch 12/113
Training batch 13/113
Training batch 14/113
Training batch 15/113
Training batch 16/113
Training batch 17/113
Training batch 18/113
Training batch 19/113
Training batch 20/113
Training batch 21/113
Training batch 22/113
Training batch 23/113
Training batch 24/113
Training batch 25/113
Training batch 26/113
Training batch 27/113
Training batch 28/113
Training batch 29/113
Training batch 30/113
Trainin